In [ ]:
import dash, requests, os, time, dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import datetime as dt
import numpy as np
from datetime import date, timedelta
from bs4 import BeautifulSoup
from dash.dependencies import Input, Output
from dotenv import load_dotenv
from dash_table.Format import Format, Scheme, Trim
from jupyter_dash import JupyterDash
load_dotenv()
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def team_data_clean(Season):
    # begin cleaning team data
    start = time.time()
    # opening both csv files into dataframes for cleaning and combining
    team1 = pd.read_csv(f'../data/raw-data/nfl-team-data-{Season}-1-raw.csv')
    team2 = pd.read_csv(f'../data/raw-data/nfl-team-data-{Season}-2-raw.csv')
    # drop all blank rows
    team1.dropna(thresh=10,inplace=True)
    team2.dropna(thresh=10,inplace=True)
    # drop useless columns
    team1.drop('LTime',axis=1,inplace=True)
    team2.drop(['LTime'],axis=1,inplace=True)
    # rename columns
    team1.rename(columns={'Tm':'Team','Unnamed: 5':'Away_Home','PF':'Points_For','PA':'Points_Against','PC':'Points_Comb',\
                         'vs. Line':'Vs_Line','Cmp':'TPass_Cmp','Att':'TPass_Att','Cmp%':'TPass_Cmp%','Yds':'TPass_Yds',\
                          'TD':'TPass_TD','Int':'TPass_Int','Sk':'TSack','Yds.1':'TSack_Yds','Rate':'TQB_Rate',\
                          'Att.1':'TRush_Att','Yds.2':'TRush_Yds','Y/A':'TRush_Y/A','TD.1':'TRush_TD','Tot':'TTot_Yds',\
                          'Ply':'TO_Play#','Y/P':'TO_Y/P','DPly':'TD_Play#','DY/P':'TD_Y/P','TO':'TTot_TO','ToP':'TO_ToP',\
                          'Time.1':'TGame_Dur','Yds.3':'TPen_Yds','OppPen':'TOpp_Pen','OppYds':'TOpp_Pen_Yds',\
                          'CombPen':'TComb_Pen','CombPenYds':'TComb_Pen_Yds','1stD':'T1st_Downs','Rsh':'T1st_by_Rsh',\
                          'Pass':'T1st_by_Pass','Pen.1':'T1st_by_Pen','3DAtt':'T3rd_Down_Att','3DConv':'T3rd_Down_Conv',\
                          '3D%':'T3rd_Down%','4DAtt':'T4th_Down_Att','4DConv':'T4th_Down_Conv','4D%':'T4th_Down%',\
                          'TD.2':'TTot_TD','XPA':'TXP_Att','XPM':'TXP_Made','FGA':'TFG_Att','FGM':'TFG_Made','2PA':'T2Pt_Att',\
                          '2PM':'T2Pt_Made','Sfty':'TSfty','Pnt':'TTimes_Punted','Yds.4':'TPunt_Yds','Y/P.1':'TPunt_Yds_Avg','Year':'Season'},inplace=True)
    team2.rename(columns={'Tm':'Team','Unnamed: 5':'Away_Home','TD':'TOpp_Tot_TD','XPA':'TOpp_XP_Att','XPM':'TOpp_XP_Made',\
                          'Att':'TOpp_FG_Att','Md':'TOpp_FG_Made','Sfty':'TOpp_Sfty','Cmp':'TOpp_Pass_Cmp','Att.1':'TOpp_Pass_Att',\
                          'Cmp%':'TOpp_Pass_Cmp%','Yds':'TOpp_Pass_Yds','TD.1':'TOpp_Pass_TD','Int':'TOpp_Pass_Int','Sk':'TOpp_Sk',\
                          'Yds.1':'TOpp_Sk_Yds','Rate':'TOpp_QB_Rate','Att.2':'TOpp_Rush_Att','Yds.2':'TOpp_Rush_Yds',\
                          'Y/A':'TOpp_Rush_Y/A','TD.2':'TOpp_Rush_TD','Tot':'TOpp_Tot_Yds','TO':'TOpp_Tot_TO',\
                          '1stDOpp':'TOpp_1st_Downs','Rush':'TOpp_1st_by_Rsh','Pass':'TOpp_1st_by_Pass','Pen':'TOpp_1st_by_Pen',\
                          'Opp3DAtt':'TOpp_3rd_Down_Att','Opp3DConv':'TOpp_3rd_Down_Conv','Opp3D%':'TOpp_3rd_Down%',\
                          'Opp4DAtt':'TOpp_4th_Down_Att','Opp4DConv':'TOpp_4th_Down_Conv','Opp4D%':'TOpp_4th_Down%',\
                          'Rush.1':'TMargin_Rush','Pass.1':'TMargin_Pass','Tot.1':'TMargin_TotYds','TO.1':'TTO_TD',\
                          'KR':'TKR_TD','PR':'TPR_TD','IR':'TInt_TD','FR':'TFmb_TD','OR':'TOtherRet_TD',\
                          'RetTD':'TAll_Ret_TD','Q1':'TMar_Thru_Q1','Q2':'TMar_Thru_Q2','Q3':'TMar_Thru_Q3',\
                          'Q1.1':'TScore_Diff_Q1','Q2.1':'TScore_Diff_Q2','Q3.1':'TScore_Diff_Q3',\
                          'Q4':'TScore_Diff_Q4','1stHalf':'TScore_Diff_1stHalf','2ndHalf':'TScore_Diff_2ndHalf','Year':'Season'},inplace=True)
    
    # merge the two dataframes based on team, season, date, time, away/home, opp, week, g#, dat, result, and ot
    team = pd.merge(left=team1,right=team2,\
                     how='outer',\
                     on=['Team','Season','Date','Time','Away_Home','Opp','Week','G#','Day','Result','OT'])
    team.set_index('Team',inplace=True)
    # drop all rows that have headers in them
    team.drop('Tm',inplace=True)
    team.reset_index(inplace=True)

    # create a list of all the column names in the team dataframe
    team_cols = []
    
    for col in team.columns:
        team_cols.append(col)

    # cleaning the away/home column to show away or home instead of @
    team.replace({'Away_Home':{'@':'Away',None:'Home'}},inplace=True)
    team[team_cols[11:]] = team[team_cols[11:]].fillna(value=0)
    # change datatype of the columns
    team[['TPass_Cmp','TPass_Att','T3rd_Down_Att','T3rd_Down_Conv','T4th_Down_Att',\
          'T4th_Down_Conv','TOpp_Pass_Cmp','TOpp_Pass_Att','TOpp_3rd_Down_Att',\
          'TOpp_3rd_Down_Conv','TOpp_4th_Down_Att','TOpp_4th_Down_Conv']] = team[['TPass_Cmp','TPass_Att','T3rd_Down_Att',\
                                                                                  'T3rd_Down_Conv','T4th_Down_Att',\
                                                                                  'T4th_Down_Conv','TOpp_Pass_Cmp',\
                                                                                  'TOpp_Pass_Att','TOpp_3rd_Down_Att',\
                                                                                  'TOpp_3rd_Down_Conv','TOpp_4th_Down_Att',\
                                                                                  'TOpp_4th_Down_Conv']].astype(float)
    # calculating the columns to show apporpriate values
    team['TPass_Cmp%'] = team['TPass_Cmp']/team['TPass_Att']
    team['T3rd_Down%'] = team['T3rd_Down_Conv']/team['T3rd_Down_Att']
    team['T4th_Down%'] = team['T4th_Down_Conv']/team['T4th_Down_Att']
    team['TOpp_Pass_Cmp%'] = team['TOpp_Pass_Cmp']/team['TOpp_Pass_Att']
    team['TOpp_3rd_Down%'] = team['TOpp_3rd_Down_Conv']/team['TOpp_3rd_Down_Att']
    team['TOpp_4th_Down%'] = team['TOpp_4th_Down_Conv']/team['TOpp_4th_Down_Att']
    # changing dates/time to datetime values
    team['Date'] = pd.to_datetime(team['Date'],errors='coerce',format='%Y-%m-%d')
    team['TGame_Dur'] = team['TGame_Dur']+':00'
    team['TO_ToP'] = '00:'+team['TO_ToP']
    team['TGame_Dur'] = pd.to_timedelta(team['TGame_Dur'],errors='coerce')
    team['TGame_Dur'] = team['TGame_Dur'].dt.total_seconds()
    team['TO_ToP'] = pd.to_timedelta(team['TO_ToP'],errors='coerce')
    team['TO_ToP'] = team['TO_ToP'].dt.total_seconds()
    #changing datatypes of columns
    team[team_cols[11:16]] = team[team_cols[11:16]].astype(float)
    team[team_cols[17]] = team[team_cols[17]].astype(float)
    team[team_cols[19:]] = team[team_cols[19:]].astype(float)
    # adding month column
    team.insert(loc=9,column='Month',value=team['Date'].dt.month)
    # chaning week datatype and removing any week larger than week 17
    team['Week'] = team['Week'].astype(int)
    team = team[team['Week']<=17]
    team['Week'] = team['Week'].astype(str)
    # saving cleaned data to csv
    team.to_csv(f'../data/clean-data/nfl-team-data-{Season}-clean.csv',index=False)
    
    end = time.time()
    print(f'Team data cleaned. {Season}',f'Time to complete: {end-start}',sep='\n')

In [ ]:
# combine data
def combine_season(Season):

    # open csv files for combining data
    inj = pd.read_csv(f'../data/clean-data/injury-report-{Season}-clean.csv')
    stat = pd.read_csv(f'../data/clean-data/player-stats-{Season}-clean.csv')
    snp = pd.read_csv(f'../data/clean-data/snap-counts-{Season}-clean.csv')
    tm = pd.read_csv(f'../data/clean-data/nfl-team-data-{Season}-clean.csv')
    
    # combine stat and snap data
    ss = pd.merge(left=stat,right=snp,how='outer',on=['Player','Season','Week'])
    ss['Team_x'] = ss['Team_x'].fillna(ss['Team_y'])
    ss['Pos_x'] = ss['Pos_y'].fillna(ss['Pos_x'])
    ss.drop(['Team_y','Pos_y'], axis=1, inplace=True)
    ss.rename(columns={'Team_x':'Team','Pos_x':'Pos'},inplace=True)
    
    # combine stat_snap data and injury data
    ssj = pd.merge(left=ss,right=inj,how='outer',on=['Player','Season','Week'])
    ssj['Date_x'] = ssj['Date_x'].fillna(ssj['Date_y'])
    ssj['Team_x'] = ssj['Team_x'].fillna(ssj['Team_y'])
    ssj['Opp_x'] = ssj['Opp_x'].fillna(ssj['Opp_y'])
    ssj.drop(['Date_y','Team_y','Opp_y','Games'],axis=1,inplace=True)
    ssj.rename(columns={'Date_x':'Date','Team_x':'Team','Opp_x':'Opp'},inplace=True)
    ssj['Player'] = ssj['Player'].str.replace('.','')
    ssj.dropna(subset=['Player'],inplace=True)
    ssj['Week'] = ssj['Week'].astype(int)
    player = ssj['Player'].unique().tolist()
    player = player*17
    player.sort()
    df2 = pd.DataFrame(player,columns=['Player'])
    df2['Season'] = Season
    df2['Week'] = df2.groupby('Player').cumcount()+1
    ssj = pd.merge(left=ssj, right=df2, how='outer', left_on=['Player', 'Season', 'Week'], right_on=['Player', 'Season', 'Week'])

    # combine team data into the individual data to fill in some blanks
    team = tm['Team'].unique().tolist()
    team = team*17
    team.sort()
    team_df = pd.DataFrame(team, columns=['Team'])
    team_df['Season'] = Season
    team_df['Opp'] = "Bye"
    team_df['Week'] = team_df.groupby('Team').cumcount()+1
    tm = pd.merge(left=tm, right=team_df, how="outer", on=['Team', 'Season', 'Week'])
    tm['Opp_x'] = tm['Opp_x'].fillna(tm['Opp_y'])
    tm.rename(columns={'Opp_x':'Opp'}, inplace=True)
    tm.drop('Opp_y', axis=1, inplace=True)
    
"""
Not merging properly because there is no "Team" value in "Team" column when merging below
"""

    ssjt = pd.merge(left=ssj, right=tm, how='outer', left_on=['Team', 'Week'], right_on=['Team', 'Week'])
    return ssjt.sort_values(['Player', 'Week']).head(100)
#     ssjt['Date_x'] = ssjt['Date_x'].fillna(ssjt['Date_y'])
#     ssjt['Away_Home_x'] = ssjt['Away_Home_x'].fillna(ssjt['Away_Home_y'])
#     ssjt['Opp_x'] = ssjt['Opp_x'].fillna(ssjt['Opp_y'])
#     ssjt['G#_x'] = ssjt['G#_x'].fillna(ssjt['G#_y'])
#     ssjt['Day_x'] = ssjt['Day_x'].fillna(ssjt['Day_y'])
#     ssjt['Result_x'] = ssjt['Result_x'].fillna(ssjt['Result_y'])
#     ssjt.drop(['Date_y','Away_Home_y','Opp_y','G#_y','Day_y','Result_y'],axis=1,inplace=True)
#     ssjt.rename(columns={'Date_x':'Date','Away_Home_x':'Away_Home','Opp_x':'Opp','G#_x':'G#','Day_x':'Day','Result_x':'Result'},inplace=True)
    
#      # sort columns for easier viewing
#     ssjt.sort_values(by=['Player','Week'],inplace=True)
    
#     # give/change all player positions to players with positions listed for the current year
#     ssjt.replace({'Pos':{'0':None}},inplace=True)
#     key = ssjt[['Player','Pos']]
#     key = key.drop_duplicates()
#     key = key.dropna()
#     key.replace({'Pos':{'DT':'DL','CB':'DB','FB':'RB','OLB':'LB','S':'DB','G':'OL','DE':'DL','FS':'DB','RT':'OL','T':'OL','RG':'OL','LT':'OL','C':'OL','LS':'LongSnap','LDE':'DL','RILB':'LB','LCB':'DB','ILB':'LB',\
#                        'MLB':'LB','TB':'RB','LDT':'DL','NT':'DL','WILL':'LB','SS':'DB','RDE':'DL','SAM':'LB','HB':'RB','WR/RS':'WR','G/T':'OL','C/G':'OL','G/C':'OL','OT':'OL','CB/RS':'DB','DB/LB':'LB','T/G':'OL',\
#                         'RB/WR':'RB','LB/FB':'LB','DE/LB':'LB','DT/DE':'DL','TE/DE':'TE','K/P':'K','OS':'OL','LG':'OL','MIKE':'LB','LILB':'LB','WLB':'LB','G/OT':'OL','SLB':'LB','RCB':'DB'}},inplace=True)
#     ssjt = pd.merge(ssjt,key,how='left',left_on='Player',right_on='Player')
#     ssjt['Pos_x'] = ssjt['Pos_y']
#     ssjt.drop(['Pos_y'],axis=1,inplace=True)
#     ssjt.rename(columns={'Pos_x':'Pos'},inplace=True)
#     ssjt['Injury'] = ssjt['Injury'].str.lower()
#     ssjt['Injury'] = ssjt['Injury'].str.strip()
#     ssjt.replace({'Injury':
#                          {'biceps':'bicep','abdominal':'abdomen','core':'abdomen',"broken fibula":'fibula','fractured forearm':'forearm',\
#                           'general medical issue':'illness','quadriceps':'quadricep','ribs':'rib','rib cage':'rib','glute':'hip',\
#                           'sprained shoulder':'sprained ac joint','thumb sprain':'thumb','nack':'neck','nack, groin':'neck, groin',\
#                           'torn acl':'acl','torn mcl':'mcl','oblique':'abdomen'}},inplace=True)

    # write data frame to csv
#     ssjt.to_csv(f'../data/clean-data/nfl-{Season}.csv',index=False)

def combine_all_season(Start,End):
    start = time.time()
    for yr in range(Start,End+1):
        combine_season(yr)
    end = time.time()
    print(f'Data clean combined and cleaned further.',f'Time to complete: {end-start}',sep='\n')

## Section 5 - Dash App

### 5.1 - Create UI From Dash

In [ ]:
def run_app(Start,End):
    inj = pd.DataFrame()
    heal = pd.DataFrame()

    for s in range(Start,End+1):
        i= pd.read_csv(f'../data/analysis-data/injury-db-{s}.csv',low_memory=False)
        h = pd.read_csv(f'../data/analysis-data/healthy-db-{s}.csv',low_memory=False)
        i['Date'] = pd.to_datetime(i['Date'],format='%Y-%m-%d')
        h['Date'] = pd.to_datetime(h['Date'],format='%Y-%m-%d')
        inj = pd.concat([inj,i])
        heal = pd.concat([heal,h])

    inj = inj[['Player','Pos','Team','Season','Fntsy_Pts','Snaps','Status','Injury','Inj_Week']]
    inj = inj.query("Snaps >= 20 and Pos == 'RB'")
    # df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

    inj = pd.pivot_table(inj,index=['Season','Injury'])
    inj.reset_index(inplace=True)
    inj.sort_values("Injury",inplace=True)

    heal = heal[['Player','Pos','Team','Season','Fntsy_Pts','Snaps','Healthy_Week']]
    heal = heal.query("Snaps >= 20 and Pos == 'RB'")
    # df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

    heal = pd.pivot_table(heal,index=['Season','Player'])
    heal.reset_index(inplace=True)
    heal.sort_values("Fntsy_Pts",ascending=False,inplace=True)

    app = JupyterDash(__name__)
#     app.layout = html.Div([
#             dash_table.DataTable(
#                 id='heal_table',
#                 columns=[{
#                     'name':i,
#                     'id':i,
#                     'type':'numeric',
#                     'format':Format(
#                         scheme=Scheme.fixed,
#                         precision=2,
#                         decimal_delimiter='.',
#                         trim=Trim.yes),
#                     'selectable': True
#                 } for i in heal.columns],
#                 sort_mode = "multi",
#                 sort_action = "native",
#                 data=heal.to_dict('records'),
#                 style_as_list_view = True,
#                 style_cell = {
#                     'padding':'5px',
#                     'minWidth':'95px'
#                 },
#                 style_header = {
#                     'backgroundColor':'#bababa',
#                     'fontWeight':'bold'
#                 },
#                 style_cell_conditional = [
#                     {
#                         'if':{'column_id':c},
#                         'textAlign':'left'
#                     } for c in heal.columns
#                 ],
#                 style_data_conditional = (
#                     [
#                         {
#                             'if':{
#                                 'row_index':'odd'
#                             },
#                             'backgroundColor':'#f5f5f5'
#                         }
#                     ] + 
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{Fntsy_Pts}} = {}'.format(i),
#                                 'column_id' : 'Fntsy_Pts'
#                             },
#                             'backgroundColor':'#14de1b'
#                         }
#                         for i in heal['Fntsy_Pts'].nlargest(5)
#                     ] +
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{{}}} <= {}'.format(col,val),
#                                 'column_id': col
#                             },
#                             'backgroundColor':'red',
#                             'color':'white'
#                         }
#                         for (col,val) in heal.quantile(0.1).iteritems()
#                     ]
#                 ),
#                 fixed_rows = {'headers':True},
#                 fixed_columns = {'headers':True,'data':1},
#                 style_table = {
#                     'height': 300,
#                     'overflowY':'auto',
#                     'overflowX':'auto',
#                     'minWidth':'100%'
#                 },
#                 page_size = 25,
#                 page_action = "native",
#                 page_current = 0,
#             ),
#             html.Div(id='datatable-interactivity-container')])
#     app.layout = html.Div([
#             dash_table.DataTable(
#                 id='inj_table',
#                 columns=[{
#                     'name':i,
#                     'id':i,
#                     'type':'numeric',
#                     'format':Format(
#                         scheme=Scheme.fixed,
#                         precision=2,
#                         decimal_delimiter='.',
#                         trim=Trim.yes),
#                     'selectable': True
#                 } for i in inj.columns],
#                 sort_mode = "multi",
#                 sort_action = "native",
#                 data=inj.to_dict('records'),
#                 style_as_list_view = True,
#                 style_cell = {
#                     'padding':'5px',
#                     'minWidth':'95px'
#                 },
#                 style_header = {
#                     'backgroundColor':'#bababa',
#                     'fontWeight':'bold'
#                 },
#                 style_cell_conditional = [
#                     {
#                         'if':{'column_id':c},
#                         'textAlign':'left'
#                     } for c in inj.columns
#                 ],
#                 style_data_conditional = (
#                     [
#                         {
#                             'if':{
#                                 'row_index':'odd'
#                             },
#                             'backgroundColor':'#f5f5f5'
#                         }
#                     ] + 
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{Fntsy_Pts}} = {}'.format(i),
#                                 'column_id' : 'Fntsy_Pts'
#                             },
#                             'backgroundColor':'#14de1b'
#                         }
#                         for i in inj['Fntsy_Pts'].nlargest(5)
#                     ] +
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{{}}} <= {}'.format(col,val),
#                                 'column_id': col
#                             },
#                             'backgroundColor':'red',
#                             'color':'white'
#                         }
#                         for (col,val) in inj.quantile(0.1).iteritems()
#                     ]
#                 ),
#                 fixed_rows = {'headers':True},
#                 fixed_columns = {'headers':True,'data':1},
#                 style_table = {
#                     'height': 300,
#                     'overflowY':'auto',
#                     'overflowX':'auto',
#                     'minWidth':'100%'
#                 },
#                 page_size = 25,
#                 page_action = "native",
#                 page_current = 0,
#             ),
#             html.Div(id='datatable-interactivity-container')])
    app.run_server(mode='inline')#,port=8060)

In [ ]:
# fantasy_socre(2016,2020)

In [ ]:
run_app(2020,2020)

In [ ]:
Start = 2016
End = 2020
inj = pd.DataFrame()
heal = pd.DataFrame()

for s in range(Start,End+1):
    i= pd.read_csv(f'../data/analysis-data/injury-db-{s}.csv',low_memory=False)
    h = pd.read_csv(f'../data/analysis-data/healthy-db-{s}.csv',low_memory=False)
    i['Date'] = pd.to_datetime(i['Date'],format='%Y-%m-%d')
    h['Date'] = pd.to_datetime(h['Date'],format='%Y-%m-%d')
    inj = pd.concat([inj,i])
    heal = pd.concat([heal,h])
inj['Fntsy_Pts'] = ((inj['IPass_Yds'] * (1/25)) +
                        (inj['IPass_TD'] * 4) +
                        (inj['IPass_Int'] * -2) +
                        (inj['IRush_Yds'] * (1/10)) +
                        (inj['IRush_TD'] * 6) +
                        (inj['IRec_Yds'] * (1/10)) +
                        (inj['IRec_TD'] * 6) +
                        (inj['IRec_Rec'] * 0.5) +
                        (inj['I2pt_Made'] * 2) +
                        (inj['IFmb_Lost'] * -2) +
                        (inj['IKR_TD'] * 6) +
                        (inj['IPR_TD'] * 6) +
                        (inj['ITack_Sk'] * 3) +
                        (inj['ITack_Solo'] * 1) +
                        (inj['ITack_Ast'] * 0.5) +
                        (inj['ITack_TFL'] * 1.5) +
                        (inj['IDef_Int'] * 3) +
                        (inj['IFmb_Forced'] * 3) +
                        (inj['IFmb_Recov'] * 3) +
                        ((inj['ITot_TD'] -
                          inj['IPass_TD'] -
                          inj['IRush_TD'] -
                          inj['IRec_TD'] -
                          inj['IKR_TD'] -
                          inj['IPR_TD']) * 6) +
                        (inj['ISfty'] * 2) +
                        (inj['IDef_PD'] * 3))

heal['Fntsy_Pts'] = ((heal['IPass_Yds'] * (1/25)) +
                        (heal['IPass_TD'] * 4) +
                        (heal['IPass_Int'] * -2) +
                        (heal['IRush_Yds'] * (1/10)) +
                        (heal['IRush_TD'] * 6) +
                        (heal['IRec_Yds'] * (1/10)) +
                        (heal['IRec_TD'] * 6) +
                        (heal['IRec_Rec'] * 0.5) +
                        (heal['I2pt_Made'] * 2) +
                        (heal['IFmb_Lost'] * -2) +
                        (heal['IKR_TD'] * 6) +
                        (heal['IPR_TD'] * 6) +
                        (heal['ITack_Sk'] * 3) +
                        (heal['ITack_Solo'] * 1) +
                        (heal['ITack_Ast'] * 0.5) +
                        (heal['ITack_TFL'] * 1.5) +
                        (heal['IDef_Int'] * 3) +
                        (heal['IFmb_Forced'] * 3) +
                        (heal['IFmb_Recov'] * 3) +
                        ((heal['ITot_TD'] -
                          heal['IPass_TD'] -
                          heal['IRush_TD'] -
                          heal['IRec_TD'] -
                          heal['IKR_TD'] -
                          heal['IPR_TD']) * 6) +
                        (heal['ISfty'] * 2) +
                        (heal['IDef_PD'] * 3))

data = pd.concat([inj,heal])
data = data[['Player','Pos','Team','Season','Week','Fntsy_Pts','Snaps','Status','Injury','Inj_Week','Healthy_Week']]

inj = inj[['Player','Pos','Team','Season','Week','Fntsy_Pts','Snaps','Status','Injury','Inj_Week']]
heal = heal[['Player','Pos','Team','Season','Week','Fntsy_Pts','Snaps','Healthy_Week']]

piv_inj = pd.pivot_table(inj,index=['Season','Injury'])
piv_inj.reset_index(inplace=True)
piv_inj.sort_values("Injury",inplace=True)

piv_heal = pd.pivot_table(heal,index=['Season','Player'])
piv_heal.reset_index(inplace=True)
piv_heal.sort_values("Fntsy_Pts",ascending=False,inplace=True)

In [ ]:
data.sort_values('Fntsy_Pts',ascending=False).head(20)